In [36]:
%load_ext autoreload

%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from spikeA.Session import *
from spikeA.Spike_train_loader import Spike_train_loader
from spikeA.Cell_group import Cell_group
from spikeA.Animal_pose import Animal_pose
from spikeA.Session import Session
from spikeA.Dat_file_reader import Dat_file_reader
from spikeA.Intervals import Intervals
from spikeA.Spike_train import Spike_train
from spikeA.Spike_waveform import Spike_waveform
from tqdm import tqdm
import os.path
import matplotlib.gridspec as gridspec
from matplotlib.backends.backend_pdf import PdfPages
import spikeA.spatial_properties
from spikeA.ttl import detectTTL
import re

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:
mouse="bt1086"
date="22022022"
name=f"{mouse}-{date}-0105"
path=f"/adata/electro/{mouse}/{name}/"

## Get a Session object and load spikes

In [44]:
ses = Kilosort_session(name=name,path=path)
ses.load_parameters_from_files()
print(ses)
stl = Spike_train_loader()
stl.load_spike_train_kilosort(ses)
cg = Cell_group(stl)

<class 'spikeA.Session.Kilosort_session'>
name = bt1086-22022022-0105
path = /adata/electro/bt1086/bt1086-22022022-0105
subject = bt1086
session_dat_time = 2022-02-22 01:05:00
fileBase = /adata/electro/bt1086/bt1086-22022022-0105/bt1086-22022022-0105
file_names = {'par': '/adata/electro/bt1086/bt1086-22022022-0105/bt1086-22022022-0105.par', 'desen': '/adata/electro/bt1086/bt1086-22022022-0105/bt1086-22022022-0105.desen', 'desel': '/adata/electro/bt1086/bt1086-22022022-0105/bt1086-22022022-0105.desel', 'sampling_rate': '/adata/electro/bt1086/bt1086-22022022-0105/bt1086-22022022-0105.sampling_rate_dat', 'stimulation': '/adata/electro/bt1086/bt1086-22022022-0105/bt1086-22022022-0105.stimulation', 'px_per_cm': '/adata/electro/bt1086/bt1086-22022022-0105/bt1086-22022022-0105.px_per_cm', 'setup': '/adata/electro/bt1086/bt1086-22022022-0105/bt1086-22022022-0105.setup', 'environmentFamiliarity': '/adata/electro/bt1086/bt1086-22022022-0105/bt1086-22022022-0105.environmentFamiliarity', 'params':

## Create a pose file 

In [45]:
if os.path.exists(f"{path}{mouse}-{date}_01.positrack"):
    pose_extension="positrack"
elif os.path.exists(f"{path}{mouse}-{date}_01.trk"):
    pose_extension="trk"
else:
    print("Position tracking extension unknown")

ap = Animal_pose(ses)
if not os.path.exists(f"{ses.fileBase}.pose.npy"):
    ap.pose_from_positrack_files(ses=ses, interpolation_frequency_hz = 50,extension=pose_extension)
    ap.save_pose_to_file() # save the file for next time
else:
    ap.load_pose_from_file()

Interpolation step: 400.0 samples
/adata/electro/bt1086/bt1086-22022022-0105/bt1086-22022022_01.dat
/adata/electro/bt1086/bt1086-22022022-0105/bt1086-22022022_01.positrack
Number of ttl pulses detected: 62251
Number of lines in positrack file: 62251
Saving /adata/electro/bt1086/bt1086-22022022-0105/bt1086-22022022_01.ttl_up.npy
hdRange: 360.03
degree to radian transformation
Invalid values: 1365
Valid values: 309890
Percentage of invalid values: 0.439%

/adata/electro/bt1086/bt1086-22022022-0105/bt1086-22022022_02.dat
/adata/electro/bt1086/bt1086-22022022-0105/bt1086-22022022_02.positrack
Number of ttl pulses detected: 30972
Number of lines in positrack file: 30972
Saving /adata/electro/bt1086/bt1086-22022022-0105/bt1086-22022022_02.ttl_up.npy
hdRange: 359.85
degree to radian transformation
Invalid values: 900
Valid values: 153960
Percentage of invalid values: 0.581%

/adata/electro/bt1086/bt1086-22022022-0105/bt1086-22022022_03.dat
/adata/electro/bt1086/bt1086-22022022-0105/bt1086-220

## Limit analysis to a particular trial

You can get the time intervals for each trial from the session object.

* Get the time at which each trial starts and ends
* Set the intervals in the spike train object. 
* Set the intervals in the animal_pose object. 

In [46]:
print(ses.desen)
if re.search('0105$', str(name)):
    sqr70_indices = [i for i, j in enumerate(ses.desen) if j == 'sqr70']
    circ80_indices = [i for i, j in enumerate(ses.desen) if j == 'circ80']

    if ses.desen[0] == 'sqr70':
        first_trial_inter = ses.trial_intervals.inter[sqr70_indices[0:1],:] # first sqr70
        last_trial_inter = ses.trial_intervals.inter[sqr70_indices[-1:],:] # last sqr70
        middle_trial_inter = ses.trial_intervals.inter[circ80_indices[0:1],:] # first circ80
    else:
        first_trial_inter = ses.trial_intervals.inter[circ80_indices[0:1],:] # first circ
        last_trial_inter = ses.trial_intervals.inter[circ80_indices[-1:],:] # last circ
        middle_trial_inter = ses.trial_intervals.inter[sqr70_indices[0:1],:] # first sqr

    print(first_trial_inter)
    print(last_trial_inter)
    print(middle_trial_inter)
else:
    first_trial_inter = ses.trial_intervals.inter[0:1,:] # no object
    second_trial_inter = ses.trial_intervals.inter[2:3,:] # object location1
    third_trial_inter = ses.trial_intervals.inter[4:5,:] # object location2
    forth_trial_inter = ses.trial_intervals.inter[6:7,:] # object location1

arena=[None]*len(ses.desen)
for n,en in enumerate(ses.desen):
    if re.search('sqr',str(ses.desen[n])):
         arena[n] = 'square'
    elif re.search('circ',str(ses.desen[n])):
        arena[n] = 'circle'
    else:
        arena[n] = ses.desen[n]
print(arena)
    
stim_trial = [i for i, j in enumerate(ses.stimulation) if j == 'MS']
if stim_trial:
    stim_trial=stim_trial[0]
    print("Stim trial:", stim_trial+1) #counting the first trial as trial 1

['circ80', 'rb', 'sqr70', 'rb', 'circ80']
[[   0.      1258.84795]]
[[3775.14    4997.19595]]
[[1888.668   3147.19195]]
['circle', 'rb', 'square', 'rb', 'circle']


In [47]:
if stim_trial:
    up_file_name = ses.path + "/" + ses.trial_names[stim_trial] + ".stim_ups_sec.npy"
    if not os.path.exists(up_file_name):
        dat_file_name = ses.path + "/" + ses.trial_names[stim_trial]+".dat"
        print(dat_file_name)
        df = Dat_file_reader(file_names=[dat_file_name],n_channels = ses.n_channels)
        laser_channel_data = df.get_data_one_block(0,df.files_last_sample[-1],np.array([ses.n_channels-2]))
        ups,downs = detectTTL(ttl_data = laser_channel_data)
        print("Number of laser pulses detected: {}".format(ups.shape[0]))
        #need to add the time that has passed before the stim trial
        ups = ups/ses.sampling_rate + ses.trial_intervals.inter[stim_trial:stim_trial+1,0] # intervals in seconds
        np.save(file = up_file_name, arr = ups)
    else:
        ups = np.load(up_file_name)

In [1]:
summary_file = f'/home/beate/summaries/{mouse}/{name}.pdf'
if not os.path.exists(f'/home/beate/summaries/{mouse}/'):
    os.mkdir(f'/home/beate/summaries/{mouse}/')
with PdfPages(summary_file) as pdf:
    for i, n in enumerate(tqdm(cg.neuron_list)):
        fig = plt.figure(figsize=(25,25), constrained_layout=False)
        layout = fig.add_gridspec(nrows=5, ncols=4, left=0.05, right=0.95, wspace=0.15, hspace=0.15, height_ratios= [10,10,10,9,1])
        n.spike_train.unset_intervals()
        
        n.spike_train.spike_time_autocorrelation(bin_size_sec=0.0005,min_sec=-0.05, max_sec=0.05)
        timestamp = n.spike_train.mid_point_from_edges(n.spike_train.st_autocorrelation_histogram[1])
        ax0 = fig.add_subplot(layout[0, 0])
        ax0.plot(timestamp, n.spike_train.st_autocorrelation_histogram[0])
        ax0.set_title(f"clu {n.name}")

        n.spike_train.spike_time_autocorrelation(bin_size_sec=0.0025,min_sec=-0.5, max_sec=0.5)
        timestamp = n.spike_train.mid_point_from_edges(n.spike_train.st_autocorrelation_histogram[1])
        ax1 = fig.add_subplot(layout[1, 0])
        ax1.plot(timestamp, n.spike_train.st_autocorrelation_histogram[0])
        
        
        ax0 = ax1 = ax2 = [None]*len([first_trial_inter,middle_trial_inter,last_trial_inter])
        for j, interv in enumerate([first_trial_inter,middle_trial_inter,last_trial_inter]):
            n.spike_train.unset_intervals()
            ap.unset_intervals()
            n.spike_train.set_intervals(interv)
            ap.set_intervals(interv)
            
            # plot firing rate maps
            n.set_spatial_properties(ap)
            n.spatial_properties.firing_rate_map_2d(cm_per_bin=2, smoothing_sigma_cm=2, smoothing=True)
            y_max=n.spatial_properties.firing_rate_map.shape[1]
            ax0[j] = fig.add_subplot(layout[0, 1+j])
            im = ax1[j].imshow(n.spatial_properties.firing_rate_map,cmap='jet')
            ax0[j].set_title(f"mRate:{n.spike_train.mean_firing_rate():.3} Hz")
            ax0[j].axis("off")
            fig.colorbar(im,ax=ax0[j])
            BS = n.spatial_properties.border_score(xy_range=np.array([[0,0],[125,125]]), arena=arena[2*j])
            plt.text(s=f"Border score: {BS:.3}", y=y_max+2, x=y_max/2-7, fontsize=12)
            
            # plot autocorrelation maps
            n.spatial_properties.firing_rate_map_2d(cm_per_bin =2, smoothing_sigma_cm =2, smoothing=True) #recalculate without pad
            n.spatial_properties.spatial_autocorrelation_map_2d()
            ax1[j] = fig.add_subplot(layout[1, 1+j])
            im = ax1[j].imshow(n.spatial_properties.spatial_autocorrelation_map, cmap='jet')
            n.spatial_properties.calculate_doughnut()
            grid_score = n.spatial_properties.grid_score()
            ax1[j].set_title("grid score:{:.3}".format(grid_score))

            # plot HD maps
            n.spatial_properties.firing_rate_head_direction_histogram(smoothing=False)
            ax2[j] = fig.add_subplot(layout[2, 1+j])
            ax2[j].plot(n.spatial_properties.firing_rate_head_direction_histo_edges[:-1],n.spatial_properties.firing_rate_head_direction_histo)  #need to cut the last for matching dimensions
            hd_score = n.spatial_properties.head_direction_score()
            ax2[j].set_title(f"HD score:{hd_score[2]:.3}, direction:{hd_score[1]:.4}°")
            ax2[j].set_xlabel("Head direction")
            ax2[j].set_ylabel("Firing rate (Hz)")
            ax2[j].set_ylim(0,np.max(n.spatial_properties.firing_rate_head_direction_histo)+1)
        
        # calculate the instaneous firing rate
        n.spike_train.unset_intervals()
        ap.unset_intervals()
        n.spike_train.instantaneous_firing_rate(bin_size_sec=20)
        # take average of consecutive values and convert from ms to min
        timepoints = 0.5*(n.spike_train.ifr[2][1:] + n.spike_train.ifr[2][:-1])/60
        timepoints = np.insert(timepoints,0,0)

        # for stacked bar chart with environments
        trial_intervals = (ses.trial_intervals.inter[n[0]][-1] for n in enumerate(ses.trial_intervals.inter))
        trial_duration = [x/60 for x in trial_intervals] # convert to min
        trial_duration.insert(0,0)
        trial_duration = [trial_duration[n+1] - trial_duration[n] for n in (range(len(trial_duration)-1))]
        colors = ['palegreen', 'honeydew', 'limegreen', 'honeydew', 'palegreen']
        labels = ses.desen # figure and axis

        ax3 = fig.add_subplot(layout[3,1:3])
        ax4 = fig.add_subplot(layout[4,1:3])
        ax3.plot(timepoints, n.spike_train.ifr[0], linewidth=2)
        ax3_2 = ax3.twinx()
        if ap.speed is None:
            ap.speed_from_pose(sigma=5)
        speed = ap.speed
        speed[speed>100] = np.nan
        time = ap.pose[:,0]/60 # in sec
        ax3_2.plot(time, speed, marker='', linewidth=1, markersize=1, color='gray', alpha=0.5)

        left = 0
        for trial,intervals in enumerate(trial_duration):
            ax4.barh(0, intervals, left = left, color=colors[trial]) #plot environments as stacked bar
            ax4.text((intervals*0.5 + left), 0, labels[trial], ha='center', weight = 'bold') # label environments
            left = left + intervals

        #ax2.set_title("Instantaneous firing rate and speed during the whole session", weight = 'bold')
        ax3.set_ylabel("Instantaneous firing rate [Hz]")
        ax4.set_xlabel("Time [min]")
        ax3_2.set_ylabel("Speed [cm/s]")
        ax3.spines['right'].set_visible(False); ax4.spines['right'].set_visible(False)
        ax4.spines['left'].set_visible(False)
        ax3.spines['top'].set_visible(False); ax4.spines['top'].set_visible(False); ax3_2.spines['top'].set_visible(False)
        ax3.spines['bottom'].set_visible(False)
        ax4.set_axisbelow(True)
        ax4.get_yaxis().set_visible(False)
        ax3.get_xaxis().set_visible(False)
        ax4.get_xaxis()
        ax3.get_shared_x_axes().join(ax4, ax3)

        
        #laser pulse - spike crosscorrelation if applicable
        if stim_trial:
            ax5 = fig.add_subplot(layout[3:5,3:4])
            min_sec=-0.03; max_sec=0.03; bin_size_sec=0.001
            n.spike_train.set_intervals(ses.trial_intervals.inter[stim_trial:stim_trial+1,:])
            hist,edges = n.spike_train.spike_time_crosscorrelation(st1=ups, st2=n.spike_train.st, bin_size_sec=bin_size_sec, min_sec=min_sec, max_sec=max_sec)
            timestamp = n.spike_train.mid_point_from_edges(edges)

            #plot
            ax5.bar(timestamp, hist, width=bin_size_sec)     
            laser_pulse = ax5.bar(0,np.max(hist),width=0.005, align='edge', color='red', alpha=0.35)
            ax5.set_xlim(min_sec, max_sec)
            ax5.set_ylim(0,np.max(hist))
            ax5.spines['top'].set_visible(False)
            ax5.spines['right'].set_visible(False)
            ax5.set_xlabel("Time [s]")
            ax5.set_ylabel("Spike count")
        
        pdf.savefig(fig)  # saves the current figure into a pdf page
        plt.show()
        plt.close()

NameError: name 'mouse' is not defined

In [49]:
def get_lfp_spectrogram(file_names, start_sample, end_sample, n_channels, channels, spec_file, sampling_rate = 20000, nperseg=2**17, overwrite=False):
    """
        Method to get the LFP spectrogram (power vs. frequency). It saves the data into a file and returns them.
        
        Arguments:
        file_names: trials to use
        start_sample, end_sample: samples from which and to which to take into account
        n_channels: number of channels in the recording
        channel: channels for which to calculate the spectrogram
        sampling rate: sampling rate of dat file (in Hz)
        nperseg: defines the resolution in the welch function
        overwrite: set True to overwrite the saved file.
        
        Return:
        array with frequency as first column and channels as other columns  
    """
    import os.path
    import numpy as np
    
    if not os.path.exists(spec_file) or overwrite==True:
        from spikeA.Dat_file_reader import Dat_file_reader
        from scipy.signal import welch
        from scipy.ndimage import gaussian_filter1d

        # read the data from the selected dat file for the selected channels
        df = Dat_file_reader(file_names=file_names, n_channels=n_channels)
        data = df.get_data_one_block(start_sample=start_sample, end_sample=end_sample, channels=np.asarray(channels))



        for i,j in enumerate(channels):
            data_1_channel = data[i,:]
            f, signal = welch(data_1_channel, fs=sampling_rate, nperseg=nperseg, scaling='spectrum')

            if i==0:
                # create an array to store the data and insert f as first column
                array = np.empty((signal.shape[0],len(channels)+1),float)
                array[:,0] = f
            array[:,i+1]=signal

        np.save(file = spec_file, arr = array)

    else:
        array = np.load(spec_file)
        
    return array

In [50]:
if stim_trial:
    n_channels=66
else:
    n_channels=65

trials = [1,3,5]
for trial in trials:
    spec_file=f"{path}spec_0{trial}.npy"
    df = Dat_file_reader(file_names=[f"{path}{mouse}-{date}_0{trial}.dat"], n_channels=n_channels)
    first, last = df.get_first_last_samples_each_file()
    array = get_lfp_spectrogram(file_names=[f"{path}{mouse}-{date}_0{trial}.dat"], start_sample=int(first), end_sample=int(last), n_channels=n_channels, channels=[n for n in range(65)], spec_file=spec_file)

In [51]:
#save the waveform
block_size=200
n_spikes=5000
overwrite=False

number_of_trials = len(ses.desen)
stim_trial = [i for i, j in enumerate(ses.stimulation) if j !='none']
if stim_trial:
    recording_channels=ses.n_channels-2
else:
    recording_channels=ses.n_channels-1
file=f"{path}{mouse}-{date}-010{number_of_trials}.waveforms.npy"
print(file)

if not os.path.exists(file) or overwrite==True:
    print(f'creating waveforms for {name}')
    df = Dat_file_reader(file_names=[f"{path}{mouse}-{date}-010{number_of_trials}.dat"], n_channels=ses.n_channels)

    #template for array:
    array=np.zeros(recording_channels*block_size*len(cg.neuron_list))
    array=np.reshape(array, (recording_channels,block_size, len(cg.neuron_list)))

    for i,n in enumerate(cg.neuron_list):
        sw = Spike_waveform(session=ses, dat_file_reader=df, spike_train=n.spike_train)
        sw.mean_waveform(block_size=block_size, channels=np.asarray(range(recording_channels)), n_spikes=n_spikes) #calculate the mean waveforms for all channels over a time interval of 200 ms
        array[:,:,i]=sw.mean_waveforms

    np.save(file = file , arr = array)
else:
    print(f"waveforms for {mouse}-{date}-010{number_of_trials} have already been created")

/adata/electro/bt1086/bt1086-22022022-0105/bt1086-22022022-0105.waveforms.npy
creating waveforms for bt1086-22022022-0105


In [2]:
summary_file = f'/home/beate/summaries/waveform/{ses.subject}/{name}.pdf'
if not os.path.exists(f'/home/beate/summaries/waveform/{ses.subject}/'):
    os.makedirs(f'/home/beate/summaries/waveform/{ses.subject}/')
    
trials = [first_trial_inter,middle_trial_inter,last_trial_inter]
if stim_trial:
    n_channels=66
else:
    n_channels=65

max_fr_neurons = np.zeros(len(cg.neuron_list))
for i, n in enumerate(cg.neuron_list):
    max_fr_neuron = np.zeros(len(trials))
    for j, interv in enumerate(trials):
        n.spike_train.unset_intervals()
        ap.unset_intervals()
        n.spike_train.set_intervals(interv)
        ap.set_intervals(interv)
        n.set_spatial_properties(ap)
        n.spatial_properties.firing_rate_map_2d(cm_per_bin =2, smoothing_sigma_cm = 2, smoothing=True)
        max_fr_neuron[j]=np.nanmax(n.spatial_properties.firing_rate_map)
    max_fr_neurons[i]=np.nanmax(max_fr_neuron)
    
# for stacked bar chart with environments
trial_intervals = (ses.trial_intervals.inter[n[0]][-1] for n in enumerate(ses.trial_intervals.inter))
trial_duration = [x/60 for x in trial_intervals] # convert to min
trial_duration.insert(0,0)
trial_duration = [trial_duration[n+1] - trial_duration[n] for n in (range(len(trial_duration)-1))]
colors = ['palegreen', 'honeydew', 'limegreen', 'honeydew', 'palegreen']
labels = ses.desen # figure and axis
        
## for waveform plot
# plot all channels in the A4x16 configuration
# create the specific location of the sites for A4x16 probes
upper_sites = [((n+1)*16-1) for n in range(4)] + [((n+1)*16-2) for n in range(4)]
ch_map = np.load(f"{path}channel_map.npy")
rows = np.tile(np.flip(range(9)), 4)
rows = np.repeat(rows,2,axis=0)
cols = (np.repeat([[1,0],[3,2],[5,4],[7,6]],9,axis=0))
cols = cols.flatten()
ax_map = [(row, col) for (row, col) in zip(rows, cols)]


with PdfPages(summary_file) as pdf:
    for i, n in enumerate(tqdm(cg.neuron_list)):
        fig = plt.figure(figsize=(25,25), constrained_layout=False)
        outer_layout = gridspec.GridSpec(4, 1, figure=fig, left=0.05, right=0.95, wspace=0.15, hspace=0.15)
        inner_layout_upper = gridspec.GridSpecFromSubplotSpec(3, 5, subplot_spec=outer_layout[0:3], width_ratios=[10,10,10,10,1.5])
        n.spike_train.unset_intervals()
        
        n.spike_train.spike_time_autocorrelation(bin_size_sec=0.0005,min_sec=-0.05, max_sec=0.05)
        timestamp = n.spike_train.mid_point_from_edges(n.spike_train.st_autocorrelation_histogram[1])
        ax0 = fig.add_subplot(inner_layout_upper[0, 0])
        ax0.plot(timestamp, n.spike_train.st_autocorrelation_histogram[0])
        ax0.set_title(f"{name}, clu {n.name}")

        n.spike_train.spike_time_autocorrelation(bin_size_sec=0.0025,min_sec=-0.5, max_sec=0.5)
        timestamp = n.spike_train.mid_point_from_edges(n.spike_train.st_autocorrelation_histogram[1])
        ax1 = fig.add_subplot(inner_layout_upper[1, 0])
        ax1.plot(timestamp, n.spike_train.st_autocorrelation_histogram[0])

        ax0 = ax1 = ax2 = [None]*len(trials)
        for j, interv in enumerate(trials):
            n.spike_train.set_intervals(interv)
            ap.set_intervals(interv)

            # plot firing rate maps
            n.set_spatial_properties(ap)
            n.spatial_properties.firing_rate_map_2d(cm_per_bin =2, smoothing_sigma_cm = 2, smoothing=True)
            if j==len(trials)-1:
                ax0[j] = fig.add_subplot(inner_layout_upper[0, 1+j:3+j])
                im = ax0[j].imshow(n.spatial_properties.firing_rate_map, cmap='jet', vmin=0, vmax=max_fr_neurons[i])
                [ax0[j].spines[d].set_visible(False) for d in ['top','right','left','bottom']]
                ax0[j].axis('off')
                #ax0[j].set_title(f"{n.spike_train.mean_firing_rate():.3} Hz", horizontalalignment='right')
                gridscore = n.spatial_properties.grid_score()
                ax0[j].set_title(f"{n.spike_train.mean_firing_rate():.3} Hz",  horizontalalignment='center', fontsize=20)
                #ax0[j].set_title(f"{gridscore:.3}", horizontalalignment='center', fontsize=20)
                fig.colorbar(im, ax=ax0[j])
            else:
                ax0[j] = fig.add_subplot(inner_layout_upper[0, 1+j])
                im = ax0[j].imshow(n.spatial_properties.firing_rate_map, cmap='jet', vmin=0, vmax=max_fr_neurons[i])
                [ax0[j].spines[d].set_visible(False) for d in ['top','right','left','bottom']]
                ax0[j].axis('off')
                ax0[j].set_title(f"{n.spike_train.mean_firing_rate():.3} Hz",  horizontalalignment='center', fontsize=20)
                gridscore = n.spatial_properties.grid_score()
                #ax0[j].set_title(f"{gridscore:.3}", horizontalalignment='center', fontsize=20)
            
            # plot autocorrelation maps
            n.spatial_properties.spatial_autocorrelation_map_2d()
            ax1[j] = fig.add_subplot(inner_layout_upper[1, 1+j])
            im = ax1[j].imshow(n.spatial_properties.spatial_autocorrelation_map, cmap='jet')
            ax1[j].set_title(f"grid score:{gridscore:.3}")

            # plot HD maps
            n.spatial_properties.firing_rate_head_direction_histogram(smoothing=False)
            ax2[j] = fig.add_subplot(inner_layout_upper[2, 1+j])
            ax2[j].plot(n.spatial_properties.firing_rate_head_direction_histo_edges[:-1],n.spatial_properties.firing_rate_head_direction_histo)  #need to cut the last for matching dimensions
            hd_score = n.spatial_properties.head_direction_score()
            ax2[j].set_title(f"HD score:{hd_score[2]:.3}, direction:{hd_score[1]:.4}°")
            ax2[j].set_xlabel("Head direction")
            ax2[j].set_ylabel("Firing rate (Hz)")
            ax2[j].set_ylim(0,np.max(n.spatial_properties.firing_rate_head_direction_histo)+1)

            
        # calculate the instaneous firing rate
        n.spike_train.unset_intervals()
        n.spike_train.instantaneous_firing_rate(bin_size_sec=20)
        # take average of consecutive values and convert from ms to min
        timepoints = 0.5*(n.spike_train.ifr[2][1:] + n.spike_train.ifr[2][:-1])/60
        timepoints = np.insert(timepoints,0,0)

        #for speed plot
        ap.unset_intervals()
        ap.speed_from_pose(sigma=5)
        time = ap.pose[:,0]/60 #convert to sec
        speed = ap.speed
        speed[speed>100] = np.nan

        # for stacked bar chart with environments
        trial_intervals = (ses.trial_intervals.inter[n[0]][-1] for n in enumerate(ses.trial_intervals.inter))
        trial_duration = [x/60 for x in trial_intervals] # convert to min
        trial_duration.insert(0,0)
        trial_duration = [trial_duration[n+1] - trial_duration[n] for n in (range(len(trial_duration)-1))]
        colors = ['palegreen', 'honeydew', 'limegreen', 'honeydew', 'limegreen', 'honeydew', 'limegreen']
        labels = ses.desen # figure and axis

        inner_layout_lower = gridspec.GridSpecFromSubplotSpec(3, 6, subplot_spec=outer_layout[3], width_ratios=[10,10,10,10,10,1.5])
        inner_layout_lower_middle = gridspec.GridSpecFromSubplotSpec(10, 1, subplot_spec=inner_layout_lower[:,2:4])
        ax3 = fig.add_subplot(inner_layout_lower_middle[0:9])
        ax4 = fig.add_subplot(inner_layout_lower_middle[9])
        
        ax3.plot(timepoints, n.spike_train.ifr[0], linewidth=2)
        ax3_2 = ax3.twinx()
        ax3_2.plot(time, speed, marker='', linewidth=1, markersize=1, color='gray', alpha=0.5)

        left = 0
        for trial,intervals in enumerate(trial_duration):
            ax4.barh(0, intervals, left = left, color=colors[trial]) #plot environments as stacked bar
            ax4.text((intervals*0.5 + left), 0, labels[trial], ha='center', weight = 'bold') # label environments
            left = left + intervals

        #ax2.set_title("Instantaneous firing rate and speed during the whole session", weight = 'bold')
        ax3.set_ylabel("Instantaneous firing rate [Hz]")
        ax4.set_xlabel("Time [min]")
        ax3_2.set_ylabel("Speed [cm/s]")
        ax3.spines['right'].set_visible(False); ax4.spines['right'].set_visible(False)
        ax4.spines['left'].set_visible(False)
        ax3.spines['top'].set_visible(False); ax4.spines['top'].set_visible(False); ax3_2.spines['top'].set_visible(False)
        ax3.spines['bottom'].set_visible(False)
        ax4.set_axisbelow(True)
        ax4.get_yaxis().set_visible(False)
        ax3.get_xaxis().set_visible(False)
        ax4.get_xaxis()
        ax3.get_shared_x_axes().join(ax4, ax3)

        
        #laser pulse - spike crosscorrelation if applicable
        if stim_trial:
            inner_layout_lower_right = gridspec.GridSpecFromSubplotSpec(1, 2, subplot_spec=inner_layout_lower[:,4])
            ax5 = fig.add_subplot(inner_layout_lower_right[:])
            min_sec=-0.03; max_sec=0.03; bin_size_sec=0.001
            n.spike_train.set_intervals(ses.trial_intervals.inter[stim_trial:stim_trial+1,:])
            hist,edges = n.spike_train.spike_time_crosscorrelation(st1=ups, st2=n.spike_train.st, bin_size_sec=bin_size_sec, min_sec=min_sec, max_sec=max_sec)
            timestamp = n.spike_train.mid_point_from_edges(edges)

            #plot
            ax5.bar(timestamp, hist, width=bin_size_sec)     
            laser_pulse = ax5.bar(0,np.max(hist),width=0.005, align='edge', color='red', alpha=0.35)
            ax5.set_xlim(min_sec, max_sec)
            ax5.set_ylim(0,np.max(hist))
            ax5.spines['top'].set_visible(False)
            ax5.spines['right'].set_visible(False)
            ax5.set_xlabel("Time [s]")
            ax5.set_ylabel("Spike count")
        
        # plot the wave forms
        inner_layout_lower_left = gridspec.GridSpecFromSubplotSpec(9, 8, subplot_spec=inner_layout_lower[:,0:2])
        n_ax = 0

        mean_waveforms=np.load(f"{path}/{mouse}-{date}-010{number_of_trials}.waveforms.npy")[:,:,i]
        # get the n_channels channels with the highest amplitudes
        n_channels = 8
        amplitudes = [abs(max(mean_waveforms[n,:])) + abs(min(mean_waveforms[n,:])) for n in range(mean_waveforms.shape[0])]
        channel_amp_list = [(c,a) for c,a in zip(amplitudes, range(mean_waveforms.shape[0]))]
        channel_amp_dict = dict(channel_amp_list)
        amp_sorted_channels = [channel_amp_dict[a] for a in sorted(channel_amp_dict)]
        channels = amp_sorted_channels[:(mean_waveforms.shape[0]-n_channels-1):-1]

        for n in range(len(ch_map)):
            if n in upper_sites:
                ax = fig.add_subplot(inner_layout_lower_left[ax_map[n_ax][0],(ax_map[n_ax][1]-1):(ax_map[n_ax][1])])
                n_ax += 2
            else:
                ax = fig.add_subplot(inner_layout_lower_left[ax_map[n_ax]])
                n_ax += 1
            if ch_map[n] in channels:
                ax.plot(mean_waveforms[np.squeeze(ch_map[n]),:], 'm')
            else:
                ax.plot(mean_waveforms[np.squeeze(ch_map[n]),:])    
            ax.spines['right'].set_visible(False)
            ax.spines['left'].set_visible(False)
            ax.spines['top'].set_visible(False)
            ax.spines['bottom'].set_visible(False)
            ax.set_ylabel(np.squeeze(ch_map[n]), rotation=0)
            ax.yaxis.set_ticks([])
            ax.set_ylim(mean_waveforms[:,:].min(), mean_waveforms[:,:].max())
            ax.get_xaxis().set_visible(False)
           
        pdf.savefig(fig)  # saves the current figure into a pdf page
        plt.show()
        plt.close()

NameError: name 'ses' is not defined

In [77]:
ses.ap = ap
ses.cg = cg
import pickle
fn = "ses2.pickle"
with open(fn,'wb') as f:
    pickle.dump(ses, f)


PicklingError: Can't pickle <class 'spikeA.Animal_pose.Animal_pose'>: it's not the same object as spikeA.Animal_pose.Animal_pose